<a href="https://colab.research.google.com/github/trinade96/Fortiate_Project/blob/master/DST_16(Random_Forest_Rules_Final).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### In this notebook we will be revisiting Mapper Plot and trying to look at customizations of it, in terms of orientation, colour and hopefully extract features of each node.

#####  **Packages Installed**

In [2]:
!pip install folium==0.2.1

     |████████████████████████████████| 71kB 2.1MB/s 
  Created wheel for folium: filename=folium-0.2.1-cp36-none-any.whl size=79979 sha256=35370c03ac2f64a6cc19158ae871abab4fd07a018bf02562715a08b39ecbc95e
  Stored in directory: /root/.cache/pip/wheels/b8/09/f0/52d2ef419c2aaf4fb149f92a33e0008bdce7ae816f0dd8f0c5
Successfully built folium
  Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3


In [3]:
!pip install coverage==3.7.1

In [4]:
!pip install pycm

     |████████████████████████████████| 61kB 2.1MB/s 
     |████████████████████████████████| 552kB 9.1MB/s 


In [5]:
!pip install scikit-tda

     |████████████████████████████████| 81kB 3.9MB/s 
     |████████████████████████████████| 102kB 4.9MB/s 
  Created wheel for ripser: filename=ripser-0.4.1-cp36-cp36m-linux_x86_64.whl size=421917 sha256=54ac7d1266e1c3c42e533590d88952ca85ae0853b6d0173240ccc6d9235850b1
  Stored in directory: /root/.cache/pip/wheels/d4/12/da/56d55c3af62ebd5e0684d521f3e58c1a85ac312502c9e2d47d
  Created wheel for hopcroftkarp: filename=hopcroftkarp-1.2.5-py2.py3-none-any.whl size=18092 sha256=653c902b18c504745a16498385baec6be3f04ac6c4a6b8f197e5933cb6ce305a
  Stored in directory: /root/.cache/pip/wheels/2b/e1/c9/1993c7f7f114b7d3fb2d3e895e02157a7ebf554861e9e54e01
Successfully built ripser hopcroftkarp


In [6]:
!pip install kmapper

In [7]:
!pip install python-igraph 

     |████████████████████████████████| 3.2MB 3.5MB/s 


##### **Packages Used**

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import kmapper as km
import sklearn
import time
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
import pycm
from pycm import *
from sklearn.cluster import DBSCAN
from sklearn import ensemble
from sklearn.decomposition import PCA
#from sklearn import datasets
#for Plotly visualization import:
from kmapper.plotlyviz import plotlyviz
from kmapper.plotlyviz import *
from kmapper.jupyter import display
import plotly.graph_objs as go

##### **Reading the data**


In [1]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/gdrive')

Drive not mounted, so nothing to flush and unmount.
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
data = pd.read_excel("/gdrive/My Drive/Random-Forest-Rules-Final.xlsx")

In [0]:
data.head()

,TRANSACTION_LOCAL_DATE,TRANSMISSION_DATE_TIME,RESPONSE_DATE_TIME,PAN,SERVICE_CODE,VALID_FROM,EXPIRY_DATE,PROC_CODE_12,PROC_CODE_34,PROC_CODE_56,PD1,PD2,PD3,PD4,PD5,PD6,PD7,PD8,PD9,PD10,PD11,PD12,TRANSACTION_AMOUNT,TRANSACTION_CURRENCY,BILLING_AMOUNT,BILLING_CURRENCY,CONVERSION_RATE_DATE,SETTLEMENT_AMOUNT,SETTLEMENT_CURRENCY,SETTLEMENT_DATE,ACQUIRING_COUNTRY_CODE,MCC,CARD_ACCEPTOR_TERM_ID,CARD_ACCEPTOR_ID,CARD_ACC_NAME_ADDRESS,AUTHORIZATION_CODE,DECISION,TARGET,C_100,C_40,C_20,ECOM_INFO,E_COM_INDICATOR,POS_ENTRY_MODE
0,21-MAR-18 11.33.24,21-MAR-18 11.33.24,21-MAR-18 11.33.24,A1586941252148110,UNK,21-AUG-17 17.10.11,01-APR-20 00.00.00,TC00,FR40,TO00,CRC1,CVC0,CCC0,ATC9,PPI4,CPI0,TCR2,TVI0,TCV0,TWC0,TOC1,PEI0,147.84,CURR356,147.84,CURR356,000,145.22,CURR356,22-MAR-18 00.00.00,CON356,MCC7399,6R00G052,9826826968,BOOK MY SHOW\\\400049\\,800204,RESULT000,0,AMI,ACS,T0784,AXASK05AEVV,ECI05,NNN
1,21-MAR-18 11.34.15,21-MAR-18 11.34.15,21-MAR-18 11.34.15,A1586941241334010,UNK,06-APR-16 20.19.15,01-APR-20 00.00.00,TC00,FR40,TO00,CRC1,CVC0,CCC0,ATC9,PPI4,CPI0,TCR2,TVI0,TCV0,TWC0,TOC1,PEI0,399.00,CURR356,399.00,CURR356,000,391.94,CURR356,22-MAR-18 00.00.00,CON356,MCC4814,6R00G052,9822047692,PAYTM RETAIL -PG-ONLINE \\NOIDA\201301\,300255,RESULT000,0,AMI,ACS,T0797,AXASK05AEVV,ECI05,NNN
2,21-MAR-18 11.36.47,21-MAR-18 11.36.47,21-MAR-18 11.36.48,A1586941049519010,UNK,03-APR-18 14.19.03,01-APR-23 00.00.00,TC00,FR40,TO00,CRC1,CVC0,CCC0,ATC9,PPI4,CPI0,TCR2,TVI0,TCV0,TWC0,TOC1,PEI0,247.20,CURR356,247.20,CURR356,000,242.82,CURR356,22-MAR-18 00.00.00,CON356,MCC7399,6R00G052,9826826968,BOOK MY SHOW\\\400049\\,700408,RESULT000,0,AMI,ACS,T0537,AXASK05AEVV,ECI05,NNN
3,21-MAR-18 11.39.44,21-MAR-18 11.39.44,21-MAR-18 11.39.44,A1586941050671000,UNK,20-APR-18 19.43.48,01-APR-23 00.00.00,TC00,FR40,TO00,CRC1,CVC0,CCC0,ATC9,PPI4,CPI0,TCR2,TVI0,TCV0,TWC0,TOC1,PEI0,359.00,CURR356,359.00,CURR356,000,352.65,CURR356,22-MAR-18 00.00.00,CON356,MCC7399,6R00G052,9826826968,BOOK MY SHOW\\\400049\\,700585,RESULT000,0,AMI,ACS,T0796,AXASK05AEVV,ECI05,NNN
4,21-MAR-18 11.43.08,21-MAR-18 11.43.08,21-MAR-18 11.43.08,A1586941257961010,UNK,30-APR-16 20.49.02,01-APR-20 00.00.00,TC00,FR40,TO00,CRC1,CVC0,CCC0,ATC9,PPI4,CPI0,TCR2,TVI0,TCV0,TWC0,TOC1,PEI0,188.00,CURR356,188.00,CURR356,000,183.81,CURR356,22-MAR-18 00.00.00,CON356,MCC5999,6R00G052,9820524155,PAYTM.COM WALLET PG \\NOIDA\201310\\,400788,RESULT000,0,AMI,ACS,T0784,AXASK05AEVV,ECI05,NNN


##### We will be redoing some portions that we did in DST 2 to get back the data in a form that can be passed on to the KMapper

In [0]:
data.shape

(105271, 44)

In [0]:
data2=data.copy()
data2.head()

,TRANSACTION_LOCAL_DATE,TRANSMISSION_DATE_TIME,RESPONSE_DATE_TIME,PAN,SERVICE_CODE,VALID_FROM,EXPIRY_DATE,PROC_CODE_12,PROC_CODE_34,PROC_CODE_56,PD1,PD2,PD3,PD4,PD5,PD6,PD7,PD8,PD9,PD10,PD11,PD12,TRANSACTION_AMOUNT,TRANSACTION_CURRENCY,BILLING_AMOUNT,BILLING_CURRENCY,CONVERSION_RATE_DATE,SETTLEMENT_AMOUNT,SETTLEMENT_CURRENCY,SETTLEMENT_DATE,ACQUIRING_COUNTRY_CODE,MCC,CARD_ACCEPTOR_TERM_ID,CARD_ACCEPTOR_ID,CARD_ACC_NAME_ADDRESS,AUTHORIZATION_CODE,DECISION,TARGET,C_100,C_40,C_20,ECOM_INFO,E_COM_INDICATOR,POS_ENTRY_MODE
0,21-MAR-18 11.33.24,21-MAR-18 11.33.24,21-MAR-18 11.33.24,A1586941252148110,UNK,21-AUG-17 17.10.11,01-APR-20 00.00.00,TC00,FR40,TO00,CRC1,CVC0,CCC0,ATC9,PPI4,CPI0,TCR2,TVI0,TCV0,TWC0,TOC1,PEI0,147.84,CURR356,147.84,CURR356,000,145.22,CURR356,22-MAR-18 00.00.00,CON356,MCC7399,6R00G052,9826826968,BOOK MY SHOW\\\400049\\,800204,RESULT000,0,AMI,ACS,T0784,AXASK05AEVV,ECI05,NNN
1,21-MAR-18 11.34.15,21-MAR-18 11.34.15,21-MAR-18 11.34.15,A1586941241334010,UNK,06-APR-16 20.19.15,01-APR-20 00.00.00,TC00,FR40,TO00,CRC1,CVC0,CCC0,ATC9,PPI4,CPI0,TCR2,TVI0,TCV0,TWC0,TOC1,PEI0,399.00,CURR356,399.00,CURR356,000,391.94,CURR356,22-MAR-18 00.00.00,CON356,MCC4814,6R00G052,9822047692,PAYTM RETAIL -PG-ONLINE \\NOIDA\201301\,300255,RESULT000,0,AMI,ACS,T0797,AXASK05AEVV,ECI05,NNN
2,21-MAR-18 11.36.47,21-MAR-18 11.36.47,21-MAR-18 11.36.48,A1586941049519010,UNK,03-APR-18 14.19.03,01-APR-23 00.00.00,TC00,FR40,TO00,CRC1,CVC0,CCC0,ATC9,PPI4,CPI0,TCR2,TVI0,TCV0,TWC0,TOC1,PEI0,247.20,CURR356,247.20,CURR356,000,242.82,CURR356,22-MAR-18 00.00.00,CON356,MCC7399,6R00G052,9826826968,BOOK MY SHOW\\\400049\\,700408,RESULT000,0,AMI,ACS,T0537,AXASK05AEVV,ECI05,NNN
3,21-MAR-18 11.39.44,21-MAR-18 11.39.44,21-MAR-18 11.39.44,A1586941050671000,UNK,20-APR-18 19.43.48,01-APR-23 00.00.00,TC00,FR40,TO00,CRC1,CVC0,CCC0,ATC9,PPI4,CPI0,TCR2,TVI0,TCV0,TWC0,TOC1,PEI0,359.00,CURR356,359.00,CURR356,000,352.65,CURR356,22-MAR-18 00.00.00,CON356,MCC7399,6R00G052,9826826968,BOOK MY SHOW\\\400049\\,700585,RESULT000,0,AMI,ACS,T0796,AXASK05AEVV,ECI05,NNN
4,21-MAR-18 11.43.08,21-MAR-18 11.43.08,21-MAR-18 11.43.08,A1586941257961010,UNK,30-APR-16 20.49.02,01-APR-20 00.00.00,TC00,FR40,TO00,CRC1,CVC0,CCC0,ATC9,PPI4,CPI0,TCR2,TVI0,TCV0,TWC0,TOC1,PEI0,188.00,CURR356,188.00,CURR356,000,183.81,CURR356,22-MAR-18 00.00.00,CON356,MCC5999,6R00G052,9820524155,PAYTM.COM WALLET PG \\NOIDA\201310\\,400788,RESULT000,0,AMI,ACS,T0784,AXASK05AEVV,ECI05,NNN


##### **Function to find categorical variables**

In [0]:
#Function to find if a column is categorical or not and assign numerical encoding to it
def check_categorical(l):
  if(len(set(l))==(len(l))):
    return False, l
  elif(len(set(l))==1):
    return False, l
  else :
    l = np.array(l)
    c=0
    for i in set(l):
      l[np.where(l == i)] = c
      c=c+1
    return True, l

##### **Splitting up categorical check and encoding function into three parts**


In [0]:
# Function that only checks categorical
'''def check_categorical(l):
   if(len(set(l))==(len(l))):
     return False
   elif(len(set(l))==1):
     return False
   else :
     return True'''

'def check_categorical(l):\n   if(len(set(l))==(len(l))):\n     return False\n   elif(len(set(l))==1):\n     return False\n   else :\n     return True'

In [0]:
# Function that assigns numeric encoding to it
def int_encoding(l):
    l = np.array(l)
    c=0
    for i in set(l):
      l[np.where(l == i)] = c
      c=c+1
    return l

In [0]:
# Function that normalizes the encoding to between 0 and 1
def normalized_encoding(l):
    l = np.array(l)
    c=0
    for i in set(l):
      l[np.where(l == i)] = c
      c=c+1
    l = l/np.max(l)
    return l

In [0]:
# Function that returns one hot encoding of a list of a categorical variable
def one_hot_encoding(l):
    l = pd.Series(l)
    one_hot = pd.get_dummies(l)
    return one_hot ## This output is a dataframe

##### All 3 encoding functions would need the check of categorical to be performed first

##### **Finding categorical variables for 'Random-Forest-Rules-Final.xlsx'**

In [0]:
names = list(data2.columns)
removable_indices =[]
for col in data2.columns:
  ## Removing columns that have NaNs
  if(data2[col].isnull().sum()>0):
    removable_indices.append(col)
  ## Removing columns that have object data type such as date object or filled with '???'
  if(data2[col].dtype.name == 'object'):
    removable_indices.append(col)

In [0]:
removable_indices = set(removable_indices)
data2 = data2.drop(removable_indices,1)

In [0]:
data2.head()

,TRANSACTION_AMOUNT,BILLING_AMOUNT,SETTLEMENT_AMOUNT,TARGET
0,147.84,147.84,145.22,0
1,399.00,399.00,391.94,0
2,247.20,247.20,242.82,0
3,359.00,359.00,352.65,0
4,188.00,188.00,183.81,0


In [0]:
data2.shape

(105271, 4)

##### It appears that too many columns get removed if we remove any columns that have any NaNs or Nulls

In [0]:
data3 = data.copy()
data3.head()

,TRANSACTION_LOCAL_DATE,TRANSMISSION_DATE_TIME,RESPONSE_DATE_TIME,PAN,SERVICE_CODE,VALID_FROM,EXPIRY_DATE,PROC_CODE_12,PROC_CODE_34,PROC_CODE_56,PD1,PD2,PD3,PD4,PD5,PD6,PD7,PD8,PD9,PD10,PD11,PD12,TRANSACTION_AMOUNT,TRANSACTION_CURRENCY,BILLING_AMOUNT,BILLING_CURRENCY,CONVERSION_RATE_DATE,SETTLEMENT_AMOUNT,SETTLEMENT_CURRENCY,SETTLEMENT_DATE,ACQUIRING_COUNTRY_CODE,MCC,CARD_ACCEPTOR_TERM_ID,CARD_ACCEPTOR_ID,CARD_ACC_NAME_ADDRESS,AUTHORIZATION_CODE,DECISION,TARGET,C_100,C_40,C_20,ECOM_INFO,E_COM_INDICATOR,POS_ENTRY_MODE
0,21-MAR-18 11.33.24,21-MAR-18 11.33.24,21-MAR-18 11.33.24,A1586941252148110,UNK,21-AUG-17 17.10.11,01-APR-20 00.00.00,TC00,FR40,TO00,CRC1,CVC0,CCC0,ATC9,PPI4,CPI0,TCR2,TVI0,TCV0,TWC0,TOC1,PEI0,147.84,CURR356,147.84,CURR356,000,145.22,CURR356,22-MAR-18 00.00.00,CON356,MCC7399,6R00G052,9826826968,BOOK MY SHOW\\\400049\\,800204,RESULT000,0,AMI,ACS,T0784,AXASK05AEVV,ECI05,NNN
1,21-MAR-18 11.34.15,21-MAR-18 11.34.15,21-MAR-18 11.34.15,A1586941241334010,UNK,06-APR-16 20.19.15,01-APR-20 00.00.00,TC00,FR40,TO00,CRC1,CVC0,CCC0,ATC9,PPI4,CPI0,TCR2,TVI0,TCV0,TWC0,TOC1,PEI0,399.00,CURR356,399.00,CURR356,000,391.94,CURR356,22-MAR-18 00.00.00,CON356,MCC4814,6R00G052,9822047692,PAYTM RETAIL -PG-ONLINE \\NOIDA\201301\,300255,RESULT000,0,AMI,ACS,T0797,AXASK05AEVV,ECI05,NNN
2,21-MAR-18 11.36.47,21-MAR-18 11.36.47,21-MAR-18 11.36.48,A1586941049519010,UNK,03-APR-18 14.19.03,01-APR-23 00.00.00,TC00,FR40,TO00,CRC1,CVC0,CCC0,ATC9,PPI4,CPI0,TCR2,TVI0,TCV0,TWC0,TOC1,PEI0,247.20,CURR356,247.20,CURR356,000,242.82,CURR356,22-MAR-18 00.00.00,CON356,MCC7399,6R00G052,9826826968,BOOK MY SHOW\\\400049\\,700408,RESULT000,0,AMI,ACS,T0537,AXASK05AEVV,ECI05,NNN
3,21-MAR-18 11.39.44,21-MAR-18 11.39.44,21-MAR-18 11.39.44,A1586941050671000,UNK,20-APR-18 19.43.48,01-APR-23 00.00.00,TC00,FR40,TO00,CRC1,CVC0,CCC0,ATC9,PPI4,CPI0,TCR2,TVI0,TCV0,TWC0,TOC1,PEI0,359.00,CURR356,359.00,CURR356,000,352.65,CURR356,22-MAR-18 00.00.00,CON356,MCC7399,6R00G052,9826826968,BOOK MY SHOW\\\400049\\,700585,RESULT000,0,AMI,ACS,T0796,AXASK05AEVV,ECI05,NNN
4,21-MAR-18 11.43.08,21-MAR-18 11.43.08,21-MAR-18 11.43.08,A1586941257961010,UNK,30-APR-16 20.49.02,01-APR-20 00.00.00,TC00,FR40,TO00,CRC1,CVC0,CCC0,ATC9,PPI4,CPI0,TCR2,TVI0,TCV0,TWC0,TOC1,PEI0,188.00,CURR356,188.00,CURR356,000,183.81,CURR356,22-MAR-18 00.00.00,CON356,MCC5999,6R00G052,9820524155,PAYTM.COM WALLET PG \\NOIDA\201310\\,400788,RESULT000,0,AMI,ACS,T0784,AXASK05AEVV,ECI05,NNN


In [0]:
data3.shape

(105271, 44)

In [0]:
data3 = data3.dropna(how='any',axis=0) 
data3.shape

(22256, 44)

##### This provides a much better solution since we get to retain all the columns but still with apparently sufficient amount of data for each

In [0]:
start_time = time.time()
names = list(data3.columns)
for col in data3.columns:
  print(col)
  bool, new_col = check_categorical(list(data3[col]))
  if(bool == True):
    ## Changing categorical columns to encoded columns
    data3[col] = new_col
data3.head()
end_time = time.time()
print("Time taken to find and encode categorical columns : {} seconds".format(end_time-start_time))

TRANSACTION_LOCAL_DATE
TRANSMISSION_DATE_TIME
RESPONSE_DATE_TIME
PAN
SERVICE_CODE
VALID_FROM
EXPIRY_DATE
PROC_CODE_12
PROC_CODE_34
PROC_CODE_56
PD1
PD2
PD3
PD4
PD5
PD6
PD7
PD8
PD9
PD10
PD11
PD12
TRANSACTION_AMOUNT
TRANSACTION_CURRENCY
BILLING_AMOUNT
BILLING_CURRENCY
CONVERSION_RATE_DATE
SETTLEMENT_AMOUNT
SETTLEMENT_CURRENCY
SETTLEMENT_DATE
ACQUIRING_COUNTRY_CODE
MCC
CARD_ACCEPTOR_TERM_ID
CARD_ACCEPTOR_ID
CARD_ACC_NAME_ADDRESS
AUTHORIZATION_CODE
DECISION
TARGET
C_100
C_40
C_20
ECOM_INFO
E_COM_INDICATOR
POS_ENTRY_MODE
Time taken to find and encode categorical columns : 50.666316747665405 seconds


##### Time taken is around 52 seconds

##### **Writing encoded dataframe to google drive to be used directly for future purposes**

In [0]:
data3.to_csv('data3.csv')
!cp data3.csv /gdrive/My\ Drive/encoded_data.csv

##### **From now on we will be using encoded_data.csv for our purposes**

In [0]:
encoded = pd.read_csv("/gdrive/My Drive/encoded_data.csv")

In [0]:
## Dropping non-numeric columns after encoding 
for col in encoded.columns:
  if(is_numeric_dtype(encoded[col])== False):
    encoded = encoded.drop(col, 1)


In [0]:
encoded.head()

,Unnamed: 0,TRANSACTION_LOCAL_DATE,TRANSMISSION_DATE_TIME,RESPONSE_DATE_TIME,PAN,VALID_FROM,EXPIRY_DATE,PROC_CODE_12,PROC_CODE_34,PD1,PD2,PD3,PD4,PD5,PD7,PD11,TRANSACTION_AMOUNT,TRANSACTION_CURRENCY,BILLING_AMOUNT,CONVERSION_RATE_DATE,SETTLEMENT_AMOUNT,SETTLEMENT_CURRENCY,ACQUIRING_COUNTRY_CODE,MCC,CARD_ACCEPTOR_TERM_ID,CARD_ACCEPTOR_ID,CARD_ACC_NAME_ADDRESS,AUTHORIZATION_CODE,DECISION,TARGET,C_100,C_40,C_20,ECOM_INFO,E_COM_INDICATOR
0,0,5852,5838,5825,7218,1024,62,0,1,0,1,0,0,0,0,0,146.0,14,146.0,0,3278.0,0,22,105,459,123,926,7143,0,0,1,0,242,5804,1
1,1,14306,14158,14119,15490,3501,62,0,1,0,1,0,0,0,0,0,398.0,14,398.0,0,5285.0,0,22,73,459,733,805,5351,0,0,1,0,181,5804,1
2,2,8607,8462,15539,13561,7165,15,0,1,0,1,0,0,0,0,0,1237.0,14,1237.0,0,1219.0,0,22,105,459,123,926,5879,0,0,1,0,504,5804,1
3,3,5948,5924,5913,4821,1991,15,0,1,0,1,0,0,0,0,0,1795.0,14,1795.0,0,1770.0,0,22,105,459,123,926,13841,0,0,1,0,565,5804,1
4,4,9301,9126,9133,4835,12169,62,0,1,0,1,0,0,0,0,0,3611.0,14,3619.0,0,3711.0,0,22,128,459,739,1054,880,0,0,1,0,242,5804,1


In [0]:
encoded.shape

(22256, 35)

In [0]:
## Dropping NaNs and object data types
for col in encoded.columns:
  if(encoded[col].isnull().sum()>0):
    encoded = encoded.drop(col,1)
  elif(encoded[col].dtype.name == 'object'):
    encoded = encoded.drop(col,1)

In [0]:
encoded.head()

,Unnamed: 0,TRANSACTION_LOCAL_DATE,TRANSMISSION_DATE_TIME,RESPONSE_DATE_TIME,PAN,VALID_FROM,EXPIRY_DATE,PROC_CODE_12,PROC_CODE_34,PD1,PD2,PD3,PD4,PD5,PD7,PD11,TRANSACTION_AMOUNT,TRANSACTION_CURRENCY,BILLING_AMOUNT,CONVERSION_RATE_DATE,SETTLEMENT_AMOUNT,SETTLEMENT_CURRENCY,ACQUIRING_COUNTRY_CODE,MCC,CARD_ACCEPTOR_TERM_ID,CARD_ACCEPTOR_ID,CARD_ACC_NAME_ADDRESS,AUTHORIZATION_CODE,DECISION,TARGET,C_100,C_40,C_20,ECOM_INFO,E_COM_INDICATOR
0,0,5852,5838,5825,7218,1024,62,0,1,0,1,0,0,0,0,0,146.0,14,146.0,0,3278.0,0,22,105,459,123,926,7143,0,0,1,0,242,5804,1
1,1,14306,14158,14119,15490,3501,62,0,1,0,1,0,0,0,0,0,398.0,14,398.0,0,5285.0,0,22,73,459,733,805,5351,0,0,1,0,181,5804,1
2,2,8607,8462,15539,13561,7165,15,0,1,0,1,0,0,0,0,0,1237.0,14,1237.0,0,1219.0,0,22,105,459,123,926,5879,0,0,1,0,504,5804,1
3,3,5948,5924,5913,4821,1991,15,0,1,0,1,0,0,0,0,0,1795.0,14,1795.0,0,1770.0,0,22,105,459,123,926,13841,0,0,1,0,565,5804,1
4,4,9301,9126,9133,4835,12169,62,0,1,0,1,0,0,0,0,0,3611.0,14,3619.0,0,3711.0,0,22,128,459,739,1054,880,0,0,1,0,242,5804,1


In [0]:
encoded.shape

(22256, 35)

##### So there was no NaNs or object data type columns in encoded.

In [0]:
## Removing the float type data columns because they may be indicative of continuous variables. Good thing the encodings preserve the data types.
## So if something was float originally so is the encoding.
for col in encoded.columns:
  if encoded[col].dtype == 'float':
    encoded = encoded.drop(col,1)

In [0]:
encoded.columns

Index(['Unnamed: 0', 'TRANSACTION_LOCAL_DATE', 'TRANSMISSION_DATE_TIME',
       'RESPONSE_DATE_TIME', 'PAN', 'VALID_FROM', 'EXPIRY_DATE',
       'PROC_CODE_12', 'PROC_CODE_34', 'PD1', 'PD2', 'PD3', 'PD4', 'PD5',
       'PD7', 'PD11', 'TRANSACTION_CURRENCY', 'CONVERSION_RATE_DATE',
       'SETTLEMENT_CURRENCY', 'ACQUIRING_COUNTRY_CODE', 'MCC',
       'CARD_ACCEPTOR_TERM_ID', 'CARD_ACCEPTOR_ID', 'CARD_ACC_NAME_ADDRESS',
       'AUTHORIZATION_CODE', 'DECISION', 'TARGET', 'C_100', 'C_40', 'C_20',
       'ECOM_INFO', 'E_COM_INDICATOR'],
      dtype='object')

In [0]:
encoded = encoded.drop('Unnamed: 0',1)

In [0]:
encoded.shape

(22256, 31)

##### **Function to create dataframe where all variables are normalized between 0 and 1**

In [0]:
## Change the range between 0 and 1
def normalize(df):
  df_new=pd.DataFrame()
  for col in df.columns:
    l = list(df[col])
    ## avoiding divisions by zero
    if np.max(l) == 0 :
      l = np.linspace(0,0,len(df[col]-1))
    else:
      l = l/np.max(l) 
    df_new[col]=l
  return df_new

In [0]:
normalize(encoded)

,TRANSACTION_LOCAL_DATE,TRANSMISSION_DATE_TIME,RESPONSE_DATE_TIME,PAN,VALID_FROM,EXPIRY_DATE,PROC_CODE_12,PROC_CODE_34,PD1,PD2,PD3,PD4,PD5,PD7,PD11,TRANSACTION_CURRENCY,CONVERSION_RATE_DATE,SETTLEMENT_CURRENCY,ACQUIRING_COUNTRY_CODE,MCC,CARD_ACCEPTOR_TERM_ID,CARD_ACCEPTOR_ID,CARD_ACC_NAME_ADDRESS,AUTHORIZATION_CODE,DECISION,TARGET,C_100,C_40,C_20,ECOM_INFO,E_COM_INDICATOR
0,0.311062,0.312293,0.311831,0.461185,0.074079,0.756098,0.0,0.5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.875,0.0,0.0,0.88,0.801527,0.541274,0.105489,0.844120,0.340289,0.0,0.0,1.0,0.0,0.389694,0.276210,0.5
1,0.760432,0.757355,0.755835,0.989713,0.253274,0.756098,0.0,0.5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.875,0.0,0.0,0.88,0.557252,0.541274,0.628645,0.733820,0.254919,0.0,0.0,1.0,0.0,0.291465,0.276210,0.5
2,0.457503,0.452659,0.831852,0.866462,0.518339,0.182927,0.0,0.5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.875,0.0,0.0,0.88,0.801527,0.541274,0.105489,0.844120,0.280072,0.0,0.0,1.0,0.0,0.811594,0.276210,0.5
3,0.316164,0.316893,0.316542,0.308031,0.144035,0.182927,0.0,0.5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.875,0.0,0.0,0.88,0.801527,0.541274,0.105489,0.844120,0.659378,0.0,0.0,1.0,0.0,0.909823,0.276210,0.5
4,0.494392,0.488178,0.488919,0.308926,0.880344,0.756098,0.0,0.5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.875,0.0,0.0,0.88,0.977099,0.541274,0.633791,0.960802,0.041923,0.0,0.0,1.0,0.0,0.389694,0.276210,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22251,0.498645,0.179576,0.179283,0.777586,0.435361,0.865854,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.875,0.0,1.0,0.88,0.557252,0.301887,0.285592,0.639927,0.188795,0.2,1.0,0.0,1.0,0.009662,0.067720,0.5
22252,0.177271,0.691880,0.984582,0.825059,0.334804,0.182927,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.875,0.0,1.0,0.88,0.687023,0.787736,0.066895,0.368277,0.188795,0.2,1.0,0.0,1.0,0.909823,0.848713,0.5
22253,0.492319,0.816626,0.815364,0.556706,0.421110,0.902439,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.875,0.0,1.0,0.88,0.801527,0.687500,0.174099,0.467639,0.188795,0.2,1.0,0.0,1.0,0.909823,0.329701,0.5
22254,0.221017,0.212849,0.937901,0.999169,0.061347,0.109756,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.875,0.0,1.0,0.88,0.557252,0.058962,0.902230,0.030082,0.188795,0.8,1.0,0.0,1.0,0.631240,0.270975,0.0


In [0]:
encoded_normalized = normalize(encoded)

##### **Function to remove columns which are a list on distinct values for eg. the index**

In [0]:
def remove_index_like(df):
  df_new = df.copy()
  indices = []
  for i in df_new.columns:
    if len(df[i]) == len(set(df[i])):
      indices.append(i)
  df_new = df_new.drop(indices,1)
  return df_new

In [0]:
encoded = remove_index_like(encoded)

In [0]:
encoded.shape

(22256, 31)

In [0]:
encoded_normalized = remove_index_like(encoded_normalized)
encoded_normalized.shape

(22256, 31)

##### There are no columns that have unique values in all rows for both encoded and encoded_normalized

##### **Function to remove constant values**


In [0]:
def remove_constants(df):
  df_new = df.copy()
  indices = []
  for col in df_new.columns:
    if len(set(df_new[col])) == 1:
      indices.append(col)
  df_new = df_new.drop(indices,1)
  return df_new

In [0]:
encoded = remove_constants(encoded)
encoded.shape

(22256, 30)

In [0]:
encoded.columns

Index(['TRANSACTION_LOCAL_DATE', 'TRANSMISSION_DATE_TIME',
       'RESPONSE_DATE_TIME', 'PAN', 'VALID_FROM', 'EXPIRY_DATE',
       'PROC_CODE_12', 'PROC_CODE_34', 'PD1', 'PD2', 'PD3', 'PD4', 'PD5',
       'PD7', 'PD11', 'TRANSACTION_CURRENCY', 'SETTLEMENT_CURRENCY',
       'ACQUIRING_COUNTRY_CODE', 'MCC', 'CARD_ACCEPTOR_TERM_ID',
       'CARD_ACCEPTOR_ID', 'CARD_ACC_NAME_ADDRESS', 'AUTHORIZATION_CODE',
       'DECISION', 'TARGET', 'C_100', 'C_40', 'C_20', 'ECOM_INFO',
       'E_COM_INDICATOR'],
      dtype='object')

### **Clustering using KMapper, pick clusters with more fraud data, using dataframe encoded**

In [0]:
encoded.to_csv('data_ready_for_clustering.csv')
!cp data_ready_for_clustering.csv /gdrive/My\ Drive/data_ready_for_clustering.csv

In [10]:
mapper_data = pd.read_csv('/gdrive/My Drive/data_ready_for_clustering.csv')
mapper_data.shape

(22256, 31)

In [11]:
mapper_data.head()

,Unnamed: 0,TRANSACTION_LOCAL_DATE,TRANSMISSION_DATE_TIME,RESPONSE_DATE_TIME,PAN,VALID_FROM,EXPIRY_DATE,PROC_CODE_12,PROC_CODE_34,PD1,PD2,PD3,PD4,PD5,PD7,PD11,TRANSACTION_CURRENCY,SETTLEMENT_CURRENCY,ACQUIRING_COUNTRY_CODE,MCC,CARD_ACCEPTOR_TERM_ID,CARD_ACCEPTOR_ID,CARD_ACC_NAME_ADDRESS,AUTHORIZATION_CODE,DECISION,TARGET,C_100,C_40,C_20,ECOM_INFO,E_COM_INDICATOR
0,0,5852,5838,5825,7218,1024,62,0,1,0,1,0,0,0,0,0,14,0,22,105,459,123,926,7143,0,0,1,0,242,5804,1
1,1,14306,14158,14119,15490,3501,62,0,1,0,1,0,0,0,0,0,14,0,22,73,459,733,805,5351,0,0,1,0,181,5804,1
2,2,8607,8462,15539,13561,7165,15,0,1,0,1,0,0,0,0,0,14,0,22,105,459,123,926,5879,0,0,1,0,504,5804,1
3,3,5948,5924,5913,4821,1991,15,0,1,0,1,0,0,0,0,0,14,0,22,105,459,123,926,13841,0,0,1,0,565,5804,1
4,4,9301,9126,9133,4835,12169,62,0,1,0,1,0,0,0,0,0,14,0,22,128,459,739,1054,880,0,0,1,0,242,5804,1


In [12]:
mapper_data = mapper_data.drop('Unnamed: 0',1)
mapper_data.shape

(22256, 30)

In [13]:
mapper_data.head()

,TRANSACTION_LOCAL_DATE,TRANSMISSION_DATE_TIME,RESPONSE_DATE_TIME,PAN,VALID_FROM,EXPIRY_DATE,PROC_CODE_12,PROC_CODE_34,PD1,PD2,PD3,PD4,PD5,PD7,PD11,TRANSACTION_CURRENCY,SETTLEMENT_CURRENCY,ACQUIRING_COUNTRY_CODE,MCC,CARD_ACCEPTOR_TERM_ID,CARD_ACCEPTOR_ID,CARD_ACC_NAME_ADDRESS,AUTHORIZATION_CODE,DECISION,TARGET,C_100,C_40,C_20,ECOM_INFO,E_COM_INDICATOR
0,5852,5838,5825,7218,1024,62,0,1,0,1,0,0,0,0,0,14,0,22,105,459,123,926,7143,0,0,1,0,242,5804,1
1,14306,14158,14119,15490,3501,62,0,1,0,1,0,0,0,0,0,14,0,22,73,459,733,805,5351,0,0,1,0,181,5804,1
2,8607,8462,15539,13561,7165,15,0,1,0,1,0,0,0,0,0,14,0,22,105,459,123,926,5879,0,0,1,0,504,5804,1
3,5948,5924,5913,4821,1991,15,0,1,0,1,0,0,0,0,0,14,0,22,105,459,123,926,13841,0,0,1,0,565,5804,1
4,9301,9126,9133,4835,12169,62,0,1,0,1,0,0,0,0,0,14,0,22,128,459,739,1054,880,0,0,1,0,242,5804,1


In [0]:
l=[]
for i  in np.arange(1,len(mapper_data),1):
  l.append(list(mapper_data.iloc[i,:]))
l=np.array(l)

In [15]:
l

array([[14306, 14158, 14119, ...,   181,  5804,     1],
       [ 8607,  8462, 15539, ...,   504,  5804,     1],
       [ 5948,  5924,  5913, ...,   565,  5804,     1],
       ...,
       [ 9262, 15266, 15231, ...,   565,  6928,     1],
       [ 4158,  3979, 17520, ...,   392,  5694,     0],
       [ 1079, 17149, 17117, ...,   565,  3488,     0]])

##### **Trials for the right parameters for clustering**

##### Here there are parameters like **nr_cubes**, inside DBScan algorithm,**eps**, **leaf_size**, **min_samples**, **metric** which are specific to problem which we try to optimize by trial. For many combinations of these parameters that are unsuitable for the dataset, the clustering would not work.

https://scikit-learn.org/stable/modules/generated/sklearn.cluster.dbscan.html

##### Look at the above link for details about arguments of DBScan

##### **This is the portion onwards that we are going to be changing**

In [0]:
'''data, labels = datasets.make_circles(n_samples=5000, noise=0.05, factor=0.3)

# Initialize
mapper = km.KeplerMapper(verbose=0)

# Fit to and transform the data
lens = mapper.fit_transform(data, projection=[0])

# Create dictionary called 'simplicial_complex' with nodes, edges and meta-information
simplicial_complex = mapper.map(lens, X=data,
                                clusterer=sklearn.cluster.DBSCAN(eps=0.1, min_samples=10),
                                cover=km.Cover(n_cubes=20, perc_overlap=0.1))'''

"data, labels = datasets.make_circles(n_samples=5000, noise=0.05, factor=0.3)\n\n# Initialize\nmapper = km.KeplerMapper(verbose=0)\n\n# Fit to and transform the data\nlens = mapper.fit_transform(data, projection=[0])\n\n# Create dictionary called 'simplicial_complex' with nodes, edges and meta-information\nsimplicial_complex = mapper.map(lens, X=data,\n                                clusterer=sklearn.cluster.DBSCAN(eps=0.1, min_samples=10),\n                                cover=km.Cover(n_cubes=20, perc_overlap=0.1))"

In [16]:
# Initialize
mapper = km.KeplerMapper(verbose=1)

# Fit to and transform the data
projected_data = mapper.fit_transform(l, projection=[0,1]) # X-Y axis

# Create dictionary called 'graph' with nodes, edges and meta-information
graph = mapper.map(projected_data, nr_cubes=25, overlap_perc = 0.1 ,clusterer = DBSCAN(algorithm ='auto',eps = 100, leaf_size = 500, metric='euclidean',
    metric_params=None, min_samples=10, n_jobs=-1, p=None))
#graph = mapper.map(projected_data, l, cover=km.Cover(n_cubes=10, perc_overlap=0.005))



KeplerMapper()
..Composing projection pipeline of length 1:
	Projections: [0, 1]
	Distance matrices: False
	Scalers: MinMaxScaler(copy=True, feature_range=(0, 1))
..Projecting on data shaped (22255, 30)

..Projecting data using: [0, 1]

..Scaling with: MinMaxScaler(copy=True, feature_range=(0, 1))

Mapping on data shaped (22255, 2) using lens shaped (22255, 2)

Creating 625 hypercubes.

Created 1550 edges and 625 nodes in 0:01:05.024310.


#####The above combination took a bit of work to get to. You may try these following algorithms for clustering that will not require you to guess as many parameters
> sklearn.cluster.KMeans(2) : You only need to specify the number of clusters you want

> sklearn.cluster.AgglomerativeClustering(2) : Same, just need to specify cluster number

> hdbscan.HBSCAN() : Hierarchical DBSCAN does not need you to guess the eps parameter of DBSCAN

In [17]:
projected_data.shape

(22255, 2)

In [18]:
# Visualize it
html=mapper.visualize(graph, path_html="Random-Forest-Rules-Final_clustering_output.html")
#display("Random-Forest-Rules-Final_clustering_output.html")

Wrote visualization to: Random-Forest-Rules-Final_clustering_output.html


##### Trying to extract some of the cluster members from the graph

In [19]:
graph['nodes'].keys()

dict_keys(['cube0_cluster0', 'cube1_cluster0', 'cube2_cluster0', 'cube3_cluster0', 'cube4_cluster0', 'cube5_cluster0', 'cube6_cluster0', 'cube7_cluster0', 'cube8_cluster0', 'cube9_cluster0', 'cube10_cluster0', 'cube11_cluster0', 'cube12_cluster0', 'cube13_cluster0', 'cube14_cluster0', 'cube15_cluster0', 'cube16_cluster0', 'cube17_cluster0', 'cube18_cluster0', 'cube19_cluster0', 'cube20_cluster0', 'cube21_cluster0', 'cube22_cluster0', 'cube23_cluster0', 'cube24_cluster0', 'cube25_cluster0', 'cube26_cluster0', 'cube27_cluster0', 'cube28_cluster0', 'cube29_cluster0', 'cube30_cluster0', 'cube31_cluster0', 'cube32_cluster0', 'cube33_cluster0', 'cube34_cluster0', 'cube35_cluster0', 'cube36_cluster0', 'cube37_cluster0', 'cube38_cluster0', 'cube39_cluster0', 'cube40_cluster0', 'cube41_cluster0', 'cube42_cluster0', 'cube43_cluster0', 'cube44_cluster0', 'cube45_cluster0', 'cube46_cluster0', 'cube47_cluster0', 'cube48_cluster0', 'cube49_cluster0', 'cube50_cluster0', 'cube51_cluster0', 'cube52_clu

In [44]:
## These are the 42 members of the cluster named 'cube0_cluster0'
graph['nodes']['cube0_cluster0']
len(graph['nodes']['cube0_cluster0'])

102

In [21]:
## We will pick up the respective columns from the dataset that we passed in for mapper and see it's features
cluster_members = pd.DataFrame(columns = mapper_data.columns)
for i in graph['nodes']['cube0_cluster0']:
  cluster_members = cluster_members.append(mapper_data.iloc[i,:])
cluster_members.shape

(102, 30)

In [22]:
cluster_members.head()

,TRANSACTION_LOCAL_DATE,TRANSMISSION_DATE_TIME,RESPONSE_DATE_TIME,PAN,VALID_FROM,EXPIRY_DATE,PROC_CODE_12,PROC_CODE_34,PD1,PD2,PD3,PD4,PD5,PD7,PD11,TRANSACTION_CURRENCY,SETTLEMENT_CURRENCY,ACQUIRING_COUNTRY_CODE,MCC,CARD_ACCEPTOR_TERM_ID,CARD_ACCEPTOR_ID,CARD_ACC_NAME_ADDRESS,AUTHORIZATION_CODE,DECISION,TARGET,C_100,C_40,C_20,ECOM_INFO,E_COM_INDICATOR
38,8230,8132,8127,7269,5283,63,0,1,0,1,0,0,0,0,0,14,0,22,93,459,916,491,19719,0,0,1,0,199,5804,1
41,1023,1037,1031,3671,4017,59,0,1,0,1,0,0,0,0,0,14,0,22,128,459,94,206,4809,0,0,1,0,438,5804,1
75,13104,13010,12970,5245,6606,82,0,1,0,1,0,0,0,0,0,14,0,22,128,459,27,807,9644,0,0,1,0,504,5804,1
90,5833,5818,5803,12911,3919,57,0,1,0,1,0,0,0,0,0,14,0,22,128,459,739,1054,8236,0,0,1,0,438,5804,1
107,11503,11402,11385,1228,6058,74,0,1,0,1,0,0,0,0,0,14,0,22,48,459,460,1082,2319,0,0,1,0,242,5804,1


In [29]:
# We will pick out those columns for which any particular value has a clear majority (more than 50% of total)
# We believe those are the characteristic features of that cluster.
for name in cluster_members.columns:
  counts = cluster_members[name].value_counts().tolist()
  if max(counts)>cluster_members.shape[0]/2 :
    print(name)

PROC_CODE_12
PROC_CODE_34
PD1
PD2
PD3
PD4
PD7
PD11
TRANSACTION_CURRENCY
SETTLEMENT_CURRENCY
ACQUIRING_COUNTRY_CODE
CARD_ACCEPTOR_TERM_ID
DECISION
TARGET
C_100
C_40
ECOM_INFO
E_COM_INDICATOR


##### So by our method these are the significant columns in determining this cluster

In [45]:
## Doing the above procedure for all clusters
start_time = time.time()
significant_overall = []
clusters = list(graph['nodes'].keys())
for c in clusters:
  #cluster_members = pd.DataFrame(columns = mapper_data.columns)
  cluster_members = mapper_data.iloc[graph['nodes'][c],:]
  significant_cols = []
  for name in cluster_members.columns:
    counts = cluster_members[name].value_counts().tolist()
    if max(counts)>cluster_members.shape[0]/2 :
      significant_cols.append(name)
  significant_overall.append(significant_cols)
  print(c)
end_time = time.time()
print("Time taken to find significant columns from clusters : {} seconds".format(end_time-start_time))

cube0_cluster0
cube1_cluster0
cube2_cluster0
cube3_cluster0
cube4_cluster0
cube5_cluster0
cube6_cluster0
cube7_cluster0
cube8_cluster0
cube9_cluster0
cube10_cluster0
cube11_cluster0
cube12_cluster0
cube13_cluster0
cube14_cluster0
cube15_cluster0
cube16_cluster0
cube17_cluster0
cube18_cluster0
cube19_cluster0
cube20_cluster0
cube21_cluster0
cube22_cluster0
cube23_cluster0
cube24_cluster0
cube25_cluster0
cube26_cluster0
cube27_cluster0
cube28_cluster0
cube29_cluster0
cube30_cluster0
cube31_cluster0
cube32_cluster0
cube33_cluster0
cube34_cluster0
cube35_cluster0
cube36_cluster0
cube37_cluster0
cube38_cluster0
cube39_cluster0
cube40_cluster0
cube41_cluster0
cube42_cluster0
cube43_cluster0
cube44_cluster0
cube45_cluster0
cube46_cluster0
cube47_cluster0
cube48_cluster0
cube49_cluster0
cube50_cluster0
cube51_cluster0
cube52_cluster0
cube53_cluster0
cube54_cluster0
cube55_cluster0
cube56_cluster0
cube57_cluster0
cube58_cluster0
cube59_cluster0
cube60_cluster0
cube61_cluster0
cube62_cluster0
cu

In [50]:
elements_in_all = list(set.intersection(*map(set, significant_overall)))
elements_in_all

['SETTLEMENT_CURRENCY',
 'PD2',
 'DECISION',
 'TRANSACTION_CURRENCY',
 'PROC_CODE_12',
 'PD1',
 'E_COM_INDICATOR',
 'ACQUIRING_COUNTRY_CODE',
 'TARGET']

##### These are the columns that come out as significant in this dataset, could  vary with data


##### **Now we move over a part where we try to mark the data using the 'TARGET' column which indicates Fraud or Not Fraud for us. The marking can be done by any other column which is categorical in nature and does not have too many categories**

In [53]:
## For our purpose we will have to remove 'TARGET' column from the data that we are passing in to mapper
mapper_data.head()

,TRANSACTION_LOCAL_DATE,TRANSMISSION_DATE_TIME,RESPONSE_DATE_TIME,PAN,VALID_FROM,EXPIRY_DATE,PROC_CODE_12,PROC_CODE_34,PD1,PD2,PD3,PD4,PD5,PD7,PD11,TRANSACTION_CURRENCY,SETTLEMENT_CURRENCY,ACQUIRING_COUNTRY_CODE,MCC,CARD_ACCEPTOR_TERM_ID,CARD_ACCEPTOR_ID,CARD_ACC_NAME_ADDRESS,AUTHORIZATION_CODE,DECISION,TARGET,C_100,C_40,C_20,ECOM_INFO,E_COM_INDICATOR
0,5852,5838,5825,7218,1024,62,0,1,0,1,0,0,0,0,0,14,0,22,105,459,123,926,7143,0,0,1,0,242,5804,1
1,14306,14158,14119,15490,3501,62,0,1,0,1,0,0,0,0,0,14,0,22,73,459,733,805,5351,0,0,1,0,181,5804,1
2,8607,8462,15539,13561,7165,15,0,1,0,1,0,0,0,0,0,14,0,22,105,459,123,926,5879,0,0,1,0,504,5804,1
3,5948,5924,5913,4821,1991,15,0,1,0,1,0,0,0,0,0,14,0,22,105,459,123,926,13841,0,0,1,0,565,5804,1
4,9301,9126,9133,4835,12169,62,0,1,0,1,0,0,0,0,0,14,0,22,128,459,739,1054,880,0,0,1,0,242,5804,1


In [56]:
mapper_data2 = mapper_data.drop('TARGET',1)
mapper_data2.head()


,TRANSACTION_LOCAL_DATE,TRANSMISSION_DATE_TIME,RESPONSE_DATE_TIME,PAN,VALID_FROM,EXPIRY_DATE,PROC_CODE_12,PROC_CODE_34,PD1,PD2,PD3,PD4,PD5,PD7,PD11,TRANSACTION_CURRENCY,SETTLEMENT_CURRENCY,ACQUIRING_COUNTRY_CODE,MCC,CARD_ACCEPTOR_TERM_ID,CARD_ACCEPTOR_ID,CARD_ACC_NAME_ADDRESS,AUTHORIZATION_CODE,DECISION,C_100,C_40,C_20,ECOM_INFO,E_COM_INDICATOR
0,5852,5838,5825,7218,1024,62,0,1,0,1,0,0,0,0,0,14,0,22,105,459,123,926,7143,0,1,0,242,5804,1
1,14306,14158,14119,15490,3501,62,0,1,0,1,0,0,0,0,0,14,0,22,73,459,733,805,5351,0,1,0,181,5804,1
2,8607,8462,15539,13561,7165,15,0,1,0,1,0,0,0,0,0,14,0,22,105,459,123,926,5879,0,1,0,504,5804,1
3,5948,5924,5913,4821,1991,15,0,1,0,1,0,0,0,0,0,14,0,22,105,459,123,926,13841,0,1,0,565,5804,1
4,9301,9126,9133,4835,12169,62,0,1,0,1,0,0,0,0,0,14,0,22,128,459,739,1054,880,0,1,0,242,5804,1


In [57]:
colouring_col = mapper_data['TARGET']
len(colouring_col)

22256

In [58]:
l2=[]
for i  in np.arange(1,len(mapper_data2),1):
  l2.append(list(mapper_data2.iloc[i,:]))
l2=np.array(l2)
l2

array([[14306, 14158, 14119, ...,   181,  5804,     1],
       [ 8607,  8462, 15539, ...,   504,  5804,     1],
       [ 5948,  5924,  5913, ...,   565,  5804,     1],
       ...,
       [ 9262, 15266, 15231, ...,   565,  6928,     1],
       [ 4158,  3979, 17520, ...,   392,  5694,     0],
       [ 1079, 17149, 17117, ...,   565,  3488,     0]])

In [59]:
l2.shape

(22255, 29)

In [60]:
# Initialize
mapper = km.KeplerMapper(verbose=1)

# Fit to and transform the data
projected_data2 = mapper.fit_transform(l2, projection=[0,1]) # X-Y axis

# Create dictionary called 'graph' with nodes, edges and meta-information
graph2 = mapper.map(projected_data2, nr_cubes=25, overlap_perc = 0.1 ,clusterer = DBSCAN(algorithm ='auto',eps = 100, leaf_size = 500, metric='euclidean',
    metric_params=None, min_samples=10, n_jobs=-1, p=None))
## All parameters are the same as before

KeplerMapper()
..Composing projection pipeline of length 1:
	Projections: [0, 1]
	Distance matrices: False
	Scalers: MinMaxScaler(copy=True, feature_range=(0, 1))
..Projecting on data shaped (22255, 29)

..Projecting data using: [0, 1]

..Scaling with: MinMaxScaler(copy=True, feature_range=(0, 1))

Mapping on data shaped (22255, 2) using lens shaped (22255, 2)

Creating 625 hypercubes.

Created 1550 edges and 625 nodes in 0:01:05.064120.


In [63]:
# Visualize it
html2=mapper.visualize(graph2, path_html="coloured_Random-Forest-Rules-Final_clustering_output.html",
                       color_function = mapper_data['TARGET'].values,
                       custom_tooltips = mapper_data['TARGET'].values)

Wrote visualization to: coloured2_Random-Forest-Rules-Final_clustering_output.html


In [62]:
mapper_data['TARGET'].value_counts()

0    21410
1      846
Name: TARGET, dtype: int64

##### So even though there are some fraud cases in the clusters, there are so few in each that the change in colour is almost imperceptible.We can try to reduce the number of clusters to see some change, or we can try to colour using a different columns which doesn't have such skewed frequency counts.

##### **Reducing number of clusters**

In [64]:
# Initialize
mapper = km.KeplerMapper(verbose=1)
# Create dictionary called 'graph' with nodes, edges and meta-information
graph3 = mapper.map(projected_data2, nr_cubes=5, overlap_perc = 0.1 ,clusterer = DBSCAN(algorithm ='auto',eps = 100, leaf_size = 500, metric='euclidean',
    metric_params=None, min_samples=10, n_jobs=-1, p=None))
## All parameters are the same as before except for 'nr_cubes'

html3=mapper.visualize(graph3, path_html="coloured2_Random-Forest-Rules-Final_clustering_output.html",
                       color_function = mapper_data['TARGET'].values,
                       custom_tooltips = mapper_data['TARGET'].values)


KeplerMapper()
Mapping on data shaped (22255, 2) using lens shaped (22255, 2)

Creating 25 hypercubes.

Created 72 edges and 25 nodes in 0:00:03.001972.
Wrote visualization to: coloured2_Random-Forest-Rules-Final_clustering_output.html


##### **Colouring using a different column**

In [76]:
for i in mapper_data.columns:
  if len(set(mapper_data[i]))<=10:
   print(i, len(set(mapper_data[i])))

PROC_CODE_12 2
PROC_CODE_34 3
PD1 4
PD2 2
PD3 2
PD4 2
PD5 5
PD7 2
PD11 2
SETTLEMENT_CURRENCY 2
DECISION 6
TARGET 2
C_100 2
C_40 3
E_COM_INDICATOR 3


##### This is important because there are 10 distinct colours in a mapper graph that can be plotted by default, there are customizations possible but only possible to run on Jupyter notebook, will add that as a seperate file.

In [77]:
html4=mapper.visualize(graph2, path_html="coloured3_Random-Forest-Rules-Final_clustering_output.html",
                       color_function = mapper_data['PD5'].values,
                       custom_tooltips = mapper_data['PD5'].values)

Wrote visualization to: coloured3_Random-Forest-Rules-Final_clustering_output.html


##### This graph is interesting because we can see a line of blue nodes in a field of yellow nodes, this is possible because there is a sufficient majority of one category in these blue nodes, and a majority of some other category in the yellow nodes. These are all categories of the variable 'PD5'. More such colours might be distinguishable when using a different palatte. We can choose to study the members of these blue nodes specifically for behaviour.

### **ROUGH WORK**


In [0]:
#mapper.visualize(graph, path_html="psycho-scores-MST.html",
                 color_function=df['var_color'].values,
                 custom_tooltips=df['var_color'].values)


In [0]:
data = np.random.random((2000, 2))
mapper = km.KeplerMapper()
lens = mapper.fit_transform(data)
graph = mapper.map(lens, data)
_ = mapper.visualize(graph, path_html="filename.html")

In [0]:
display("filename.html")

In [0]:
!ls '/gdrive/My Drive'